# Human-in-the-loop（HITL）是什麼？

Human-in-the-loop 的作用是讓人類參與決策，補充或修正系統狀態：
> 每個 HITL 接收 state → 向使用者詢問或請使用者確認 → 回傳要更新的 state 部分欄位。


# Interrupt

LangGraph 為 HITL 設計了一個節點 Interrupt。作用是在流程中途暫停或改變執行，根據條件觸發特定動作。
> interrupt：讓 Graph 停下來，直到人類傳入新的 state 再繼續往下跑

## 基本用法
### 1. 定義 State 和 Node


In [25]:
from typing import TypedDict, Optional
from langgraph.types import interrupt

class AgentState(TypedDict):
    approved: Optional[bool]

def approval_node(state: AgentState):
    """
    範例 Node：請使用者確認操作是否批准
    """
    print("暫停程式")
    approved = interrupt("Do you approve this action? (yes/no)")
    print("繼續執行，回傳值：",approved)
    return {"approved": approved}

說明：

- interrupt() 會暫停 graph 執行，等待外部輸入
- 參數是提示訊息，會顯示給使用者
- 返回值是使用者的輸入

### 2. 建立和執行 Graph
執行時，需要 MemorySaver（後面會有章節詳細介紹 Memory 管理）：

In [26]:
from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command

# 建立 graph
workflow = StateGraph(AgentState)
workflow.add_node("approval", approval_node)
workflow.set_entry_point("approval")
workflow.set_finish_point("approval")

checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "1"}}

result = graph.invoke({"approved": None}, config)

user_input = input("Do you approve this action? (yes/no): ")
approved = user_input.lower() == "yes" # 將使用者輸入轉換為布林值

result = graph.invoke(
    Command(resume=approved),  # 將使用者的回應傳入
    config
)


暫停程式
暫停程式
繼續執行，回傳值： False



流程：

1. 第一次執行：graph 跑到 interrupt() 時會停止執行
2. 儲存狀態：checkpointer 把當前狀態存起來
3. 等待輸入：跳到你寫的user_input，等你把結果傳回去
4. 第二次執行：用同個 thread_id 再次呼叫，重新執行該 node
5. 當執行到interrupt，會使用resume替換內容


說明：
- config：包含 thread_id，用來識別對話並存取狀態
- checkpointer：儲存 graph 狀態的工具，interrupt 必須搭配使用
- Command(resume=...)：在 graph 因 interrupt 暫停後，用來將外部輸入傳回並**重新執行這個節點**(不是從interrupt繼續)
    - 小心有坑 ↓

### API 呼叫重複執行問題

In [ ]:
# 錯誤寫法
def my_node(state):
    api_call()  # 這會在恢復執行時重複呼叫
    answer = interrupt("確認？")
    return {"data": answer}

# 正確寫法 1：放在 interrupt 之後
def my_node(state):
    answer = interrupt("確認？")
    api_call(answer)  # OK
    return {"data": answer}

# 正確寫法 2：拆成兩個節點
def approval_node(state):
    answer = interrupt("確認？")
    return {"answer": answer}

def api_node(state):
    api_call()  # OK，在獨立節點中
    return state

其他相關的坑請參閱：https://docs.langchain.com/oss/python/langgraph/interrupts#rules-of-interrupts

## 根據結果跳轉到不同 Node

In [1]:
from typing import TypedDict, Optional, Literal
from langgraph.types import interrupt

class AgentState(TypedDict):
    approved: Optional[bool]
    message: Optional[str]

def approval_node(state: AgentState):
    """
    範例 Node:請使用者確認操作是否批准
    """
    print("=== 進入審批節點 ===")
    print("暫停程式,等待使用者輸入...")
    approved = interrupt("Do you approve this action? (yes/no)")
    print(f"繼續執行,收到回傳值:{approved}")
    return {"approved": approved}

def route_decision(state: AgentState) -> Literal["approve_action", "reject_action"]:
    """
    路由函數:根據 approved 狀態決定下一個節點
    """
    print("=== 進入路由決策節點 ===")
    if state.get("approved"):
        print("路由至:批准操作節點")
        return "approve_action"
    else:
        print("路由至:拒絕操作節點")
        return "reject_action"

def approve_action(state: AgentState):
    """
    批准操作節點
    """
    print("\n" + "="*50)
    print("操作已批准!")
    print("="*50)
    print("正在執行批准後的操作...")
    print("- 步驟 1: 驗證權限")
    print("- 步驟 2: 執行操作")
    print("- 步驟 3: 記錄日誌")
    print("操作完成!")
    return {"message": "操作已成功執行"}

def reject_action(state: AgentState):
    """
    拒絕操作節點
    """
    print("\n" + "="*50)
    print("操作已拒絕!")
    print("="*50)
    print("操作被取消,原因:使用者未批准")
    print("- 記錄拒絕日誌")
    print("- 清理暫存資料")
    print("流程已終止")
    return {"message": "操作已被取消"}

In [2]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command

# 建立 graph
workflow = StateGraph(AgentState)

# 新增所有節點
workflow.add_node("approval", approval_node)
workflow.add_node("approve_action", approve_action)
workflow.add_node("reject_action", reject_action)

# 設定入口點
workflow.set_entry_point("approval")

# 新增條件路由
workflow.add_conditional_edges(
    "approval",
    route_decision,
    {
        "approve_action": "approve_action",
        "reject_action": "reject_action"
    }
)

# 設定結束點
workflow.add_edge("approve_action", END)
workflow.add_edge("reject_action", END)

# 編譯 graph
checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

config = {"configurable": {"thread_id": "1"}}

result = graph.invoke({"approved": None}, config)

user_input = input("Do you approve this action? (yes/no): ")
approved = user_input.lower() == "yes" # 將使用者輸入轉換為布林值

result = graph.invoke(
    Command(resume=approved),  # 將使用者的回應傳入
    config
)

=== 進入審批節點 ===
暫停程式,等待使用者輸入...
=== 進入審批節點 ===
暫停程式,等待使用者輸入...
繼續執行,收到回傳值:True
=== 進入路由決策節點 ===
路由至:批准操作節點

操作已批准!
正在執行批准後的操作...
- 步驟 1: 驗證權限
- 步驟 2: 執行操作
- 步驟 3: 記錄日誌
操作完成!
